In [1]:
#exec(open("gpu_par.py").read())


2020-10-14 11:42:54,284	INFO resource_spec.py:231 -- Starting Ray with 954.79 GiB memory available for workers and up to 186.26 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-14 11:42:54,792	INFO services.py:1193 -- View the Ray dashboard at localhost:8265


In [1]:
import ray
import os

ray.init() #if want to use less gpus - specify here

def get_gpu_id():
    try:
        gpu_id = ray.get_gpu_ids()[0]
    except:
        gpu_id = 0
    return gpu_id

2020-10-14 22:51:44,001	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265


In [2]:
ray_objs = []

def gpu_map(f, *args):
    @ray.remote(num_cpus=1, num_gpus=4)
    def f_gpu(*args):
        os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
        return f(*args)

    objs = [f_gpu.remote(*x) for x in zip(*args)]
    ray_objs.extend(objs)
    out = [ray.get(x) for x in objs]
    return out

def kill_gpu_processes():
    [ray.cancel(x) for x in ray_objs]

In [6]:
kill_gpu_processes()

In [4]:
ray.get_gpu_ids()

[]

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]

KeyError: 'CUDA_VISIBLE_DEVICES'

In [3]:
import torch
def f(n):
    return torch.rand(n)

In [4]:
@ray.remote(num_cpus=1, num_gpus=2)
def rf(*args):
    return f(args)

In [5]:
o = rf.remote(4)

In [6]:
ray.get(o)

tensor([0.1762, 0.4989, 0.0677, 0.3453])

In [28]:
gpu_map(f, [2,3,400000000])

[tensor([0.9398, 0.1469]),
 tensor([0.3973, 0.1522, 0.5628]),
 tensor([0.5736, 0.6767, 0.0189,  ..., 0.0179, 0.7656, 0.0348])]

In [29]:
ray_objs

[ObjectRef(cb230a572350ff44ffffffff0100000001000000),
 ObjectRef(7bbd90284b71e599ffffffff0100000001000000),
 ObjectRef(bd37d2621480fc7dffffffff0100000001000000),
 ObjectRef(88866c7daffdd00effffffff0100000001000000),
 ObjectRef(d251967856448cebffffffff0100000001000000),
 ObjectRef(3bf0c856ace5a4d8ffffffff0100000001000000),
 ObjectRef(72e11b46e93d91e4ffffffff0100000001000000),
 ObjectRef(62223d85d5e7cd76ffffffff0100000001000000),
 ObjectRef(3106d80c4e3c2369ffffffff0100000001000000),
 ObjectRef(ae935fc0cb63c7d2ffffffff0100000001000000),
 ObjectRef(3db7cfefbaf00fd0ffffffff0100000001000000),
 ObjectRef(be3cb809d884a562ffffffff0100000001000000),
 ObjectRef(bd5c5340e4cc89c2ffffffff0100000001000000),
 ObjectRef(7f10737098927148ffffffff0100000001000000)]

In [17]:
obj_ref = ray.put(1)
ray.get(obj_ref)

1

In [26]:
@ray.remote(num_gpus=4)
def use_gpu():
    print("ray.get_gpu_ids(): {}".format(ray.get_gpu_ids()))
    print("CUDA_VISIBLE_DEVICES: {}".format(os.environ["CUDA_VISIBLE_DEVICES"]))

In [27]:
use_gpu.remote()

ObjectRef(87b4f726eda7082bffffffff0100000001000000)

(pid=4915) ray.get_gpu_ids(): [3, 2, 1, 0]
(pid=4915) CUDA_VISIBLE_DEVICES: 3,2,1,0
